In [1]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "your key"
import openai
openai.api_key = "your key"

In [3]:
from pathlib import Path
from llama_index.core import download_loader, VectorStoreIndex, load_index_from_storage, StorageContext
#from llama_index.storage.storage_context import StorageContext

PDFReader = download_loader("PDFReader")

loader = PDFReader()

class PodcastTitle:
    def __init__(self, name, about, file, key):
        self.name = name
        self.about = about   
        self.file = file
        self.key = key

podcast_titles = [
    PodcastTitle("10 Tools for Managing Stress and Anxiety with Huberman", "manage stress and anxiety", "behaviour/10_Tools_for_Managing_Stress_&_Anxiety_Huberman_Lab_Podcast_10.pdf", "tools_for_stress"),
    PodcastTitle("The Science of Setting and Achieving Goals with Huberman", "set and achieve goals", "behaviour/55_The_Science_of_Setting_&_Achieving_Goals_Huberman_Lab_Podcast_55.pdf", "setting_goals"),
    PodcastTitle("Dr Chris Palmer Diet and Nutrition for Mental Health with Huberman", "have healthy diet for mental health", "food/99_Dr_Chris_Palmer_Diet_&_Nutrition_for_Mental_Health_Huberman_Lab_Podcast_99.pdf", "diet_nutrition"),
]

podcast_vector_index = {}
for podcast in podcast_titles:
    try:
        storage_context = StorageContext.from_defaults(persist_dir=f"./storage/cache/{podcast.key}_vector")
        podcast_vector_index[podcast.key] = load_index_from_storage(storage_context)
    except:
        documents = loader.load_data(file=Path(f"./assets/AndrewHuberman/{podcast.file}"))
        vector_index = VectorStoreIndex.from_documents(documents)
        podcast_vector_index[podcast.key] = vector_index
        vector_index.storage_context.persist(persist_dir=f"./storage/cache/{podcast.key}_vector")




C:\Users\Boris\AppData\Local\Temp\ipykernel_19044\2881837558.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  PDFReader = download_loader("PDFReader")


INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.


In [4]:
podcast_vector_index

{'tools_for_stress': <llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x25a5bf3bf50>,
 'setting_goals': <llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x25a5bf801d0>,
 'diet_nutrition': <llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x25a5c7db050>}

In [8]:
from llama_index.core.postprocessor import KeywordNodePostprocessor
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata


node_processor = KeywordNodePostprocessor(
    exclude_keywords=["supplements", "LMNT", "InsideTracker", "Helix", "ROKA", "Athletic Greens", "Thesis", "Eight Sleep"]
)

query_engine_tools = []
podcats_vector_engines = {}

for podcast in podcast_titles:
    retriever = VectorIndexRetriever(
        index = podcast_vector_index[podcast.key],
        similarity_top_k=3,
    )

    podcats_vector_engines[podcast.key] = RetrieverQueryEngine(
        retriever = retriever,
        node_postprocessors=[node_processor]
    )

    new_tool = QueryEngineTool(
        query_engine=podcats_vector_engines[podcast.key],
        metadata = ToolMetadata(
            name=f"{podcast.key}_vector_tool",
            description=f"Useful for retrieving specific context from a podcast {podcast.name}. "
            f"Use when you need information related to {podcast.about}.",
        )

    )
    query_engine_tools.append(new_tool)

In [9]:
query_engine_tools

In [16]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

agent = OpenAIAgent.from_tools(query_engine_tools, llm=OpenAI(temperature=0, model="gpt-3.5-turbo"), verbose=True)

In [17]:
openai.log = "debug"

In [18]:
response = agent.chat("Tell me about Ketogenic diet")

Added user message to memory: Tell me about Ketogenic diet
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: diet_nutrition_vector_tool with args: {"input":"Ketogenic diet"}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Got output: The ketogenic diet involves reducing carbohydrate intake and increasing the consumption of fats, which leads the body to enter a state of ketosis where it burns fat for energy. This diet has been shown to have various effects on metabolism, including lowering glucose levels, improving insulin signaling, and increasing 

In [20]:
response.sources

[ToolOutput(content='The ketogenic diet involves reducing carbohydrate intake and increasing the consumption of fats, which leads the body to enter a state of ketosis where it burns fat for energy. This diet has been shown to have various effects on metabolism, including lowering glucose levels, improving insulin signaling, and increasing mitochondrial biogenesis. Ketones produced during ketosis are essential for fueling the brain. While some individuals may benefit from a strict ketogenic diet, others may find it challenging to adhere to. In such cases, exogenous ketones in the form of supplements have been considered, but there is a lack of conclusive trial data on their effectiveness compared to the actual ketogenic diet.', tool_name='diet_nutrition_vector_tool', raw_input={'input': 'Ketogenic diet'}, raw_output=Response(response='The ketogenic diet involves reducing carbohydrate intake and increasing the consumption of fats, which leads the body to enter a state of ketosis where it